In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 15.8 MB/s 
     |████████████████████████████████| 596 kB 55.1 MB/s 
     |████████████████████████████████| 6.6 MB 48.3 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 60.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [1]:
import pandas as pd
import numpy as np
import json
import os 

import torch.nn as nn
import torch
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup

### Change the path here

In [2]:
training_filepath = '../../Data/BioASQ_data/BioASQ-training7b/trainining7b.json'
test_directory = '../../Data/BioASQ_data/Task7BGoldenEnriched/'

In [3]:
training_text = []
test_text = []

# Filter out the training data
with open (training_filepath, "r") as f:
    data = json.loads(f.read())
    texts = data['questions']
    for text in texts:
        if 'exact_answer' in text.keys():
            if text['exact_answer'] == 'yes' or text['exact_answer'] == 'no':
                training_text.append(text)

# Filter out the text data
directory = test_directory
for filename in os.listdir(directory):
    with open (directory+filename, "r") as f:
        data = json.loads(f.read())
        texts = data['questions']
        for text in texts:
            if 'exact_answer' in text.keys():
                if text['exact_answer'] == 'yes' or text['exact_answer'] == 'no':
                    test_text.append(text)

In [4]:
def process_data(texts):
    question_list = []
    context_list = []
    target_list = []
    for text in texts:
        question_list.append(text['body'])
        context_list.append(' '. join([x['text'] for x in text['snippets']]))
        target_list.append(1 if text['exact_answer'] == 'yes' else 0)
    df = pd.DataFrame(zip(question_list, context_list, target_list), columns=['question', 'context', 'target'])
    return df

In [5]:
class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained('/home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/4GB-checkpoints/model-trained-0-3531-4GB')
        self.out = nn.Linear(768, 2)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, ids, mask, token_type_ids):
        _, output = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids, return_dict=False)
        output = self.out(output)
        return self.softmax(output)

class BERTDatasetTraining:
    def __init__(self, question, context, target, tokenizer, max_len):
        self.question = question
        self.context = context
        self.target = target
        
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __len__(self):
        return len(self.question)
    
    def __getitem__(self, item):
        question= str(self.question[item])
        context = str(self.context[item])

        inputs = self.tokenizer.encode_plus(
            question,
            context,
            add_special_tokens=True
        )

        ids = inputs['input_ids']
        token_type_ids = inputs['token_type_ids']
        mask = inputs['attention_mask']
        
        padding_len = self.max_len - len(ids)
        
        ids = ids[:self.max_len] + ([0] * padding_len) 
        token_type_ids = token_type_ids[:self.max_len] + ([0] * padding_len)
        mask = mask[:self.max_len] + ([0] * padding_len)
        

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(self.target[item], dtype=torch.long)
        }


def loss_fn(outputs, target):
    return nn.CrossEntropyLoss()(outputs, target)


def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()
    for bi, d in enumerate(data_loader):
        ids = d['ids']
        mask = d['mask']
        token_type_ids = d['token_type_ids']
        target = d['target']
        
        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        target = target.to(device, dtype=torch.long)
        
        optimizer.zero_grad()
        outputs = model(ids, mask, token_type_ids)

        loss = loss_fn(outputs, target)
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        if bi % 10 == 0:
            print(f'bi={bi}, loss={loss}')

            
def eval_loop_fn(data_loader, model, device):
    model.eval()
    fin_targets = []
    fin_outputs = []
    for bi, d in enumerate(data_loader):
        with torch.no_grad():
            ids = d['ids'].to(device, dtype=torch.long)
            mask = d['mask'].to(device, dtype=torch.long)
            token_type_ids = d['token_type_ids'].to(device, dtype=torch.long)
            target = d['target'].to(device, dtype=torch.long)
          
            outputs = model(ids, mask, token_type_ids)
          
            fin_targets.append(target.cpu().detach().numpy())
            fin_outputs.append(outputs.cpu().detach().numpy())

    return np.vstack(fin_outputs), np.hstack(fin_targets)

In [8]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
TEST_BATCH_SIZE = 4
EPOCHS = 4
LEARNING_RATE = 3e-5
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

tokenizer = transformers.BertTokenizer.from_pretrained('../../Preprocessing/Tokenization/wp-vocab-30500-vocab.txt')
print(tokenizer)
model = BERTBaseUncased().to(device)

train_df = process_data(training_text)
test_df = process_data(test_text)

train_dataset = BERTDatasetTraining(
    question=train_df.question.values,
    context=train_df.context.values,
    target=train_df.target.values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)
train_data_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True
)

test_dataset = BERTDatasetTraining(
    question=test_df.question.values,
    context=test_df.context.values,
    target=test_df.target.values,
    tokenizer=tokenizer,
    max_len=MAX_LEN
)
test_data_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=TEST_BATCH_SIZE,
    shuffle=True,
    drop_last=True
)

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
num_training_steps = int(len(train_dataset) / TRAIN_BATCH_SIZE * EPOCHS)
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

for epoch in range(EPOCHS):
    train_loop_fn(train_data_loader, model, optimizer, device, scheduler)
    output, target = eval_loop_fn(test_data_loader, model, device)
    acc = (output.argmax(1) == target).sum() / len(target)
    print(f'epoch: {epoch}, acc: {acc}')
    # print(output.tolist())
    # print(target.tolist())
    print('---------------------')

/anaconda/envs/py38_pytorch/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1658: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


PreTrainedTokenizer(name_or_path='../../Preprocessing/Tokenization/wp-vocab-30500-vocab.txt', vocab_size=30500, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


Some weights of the model checkpoint at /home/americanthinker/notebooks/pytorch/NationalSecurityBERT/Modeling/checkpoints/4GB-checkpoints/model-trained-0-3531-4GB were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at /h

bi=0, loss=0.7468404769897461
bi=10, loss=0.8125408291816711
bi=20, loss=0.5632827877998352
bi=30, loss=0.43850085139274597
bi=40, loss=0.6881063580513
bi=50, loss=0.4384167790412903
bi=60, loss=0.43840497732162476
bi=70, loss=0.4384075701236725
bi=80, loss=0.438368558883667
bi=90, loss=0.43836554884910583
epoch: 0, acc: 0.6714285714285714
---------------------
bi=0, loss=0.5632815361022949
bi=10, loss=0.6881755590438843
bi=20, loss=0.5632649064064026
bi=30, loss=0.563271701335907
bi=40, loss=0.31340697407722473
bi=50, loss=0.4383338987827301
bi=60, loss=0.4383391737937927
bi=70, loss=0.6881988644599915
bi=80, loss=0.31339719891548157
bi=90, loss=0.8131579160690308
epoch: 1, acc: 0.6714285714285714
---------------------
bi=0, loss=0.6882166266441345
bi=10, loss=0.8131458759307861
bi=20, loss=0.43832942843437195
bi=30, loss=0.6882184743881226
bi=40, loss=0.31337735056877136
bi=50, loss=0.43832165002822876
bi=60, loss=0.5632729530334473
bi=70, loss=0.4383278489112854
bi=80, loss=0.438320

In [9]:
output, target = eval_loop_fn(test_data_loader, model, device)

In [10]:
acc = (output.argmax(1) == target).sum() / len(target)

In [11]:
acc

0.6714285714285714